In [1]:
%load_ext autoreload
%autoreload 2

In [116]:
import numpy as np
import pandas as pd
import sklearn
from utils import _ohlc,df_filter_times
from sklearn.cluster import KMeans
import mplfinance as mpf
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# Getting clusters

In [1]:
df_all = pd.read_csv("./data/1m-10d.csv",parse_dates=["datetime"])
df_market = df_filter_times(df_all)

NameError: name 'pd' is not defined

In [162]:
df = df_market[df_market['name'].isin(["NFLX","GOOG","AAPL"])]
df

,open,high,low,close,volume,datetime,name
475556,154.010,155.2200,153.990,155.140,427851,2024-04-08 09:30:00-04:00,GOOG
475557,155.070,155.2399,154.880,154.930,76196,2024-04-08 09:31:00-04:00,GOOG
475558,154.960,155.1200,154.590,155.110,51774,2024-04-08 09:32:00-04:00,GOOG
475559,155.090,155.5980,154.980,155.430,86735,2024-04-08 09:33:00-04:00,GOOG
475560,155.450,155.7350,155.320,155.338,63866,2024-04-08 09:34:00-04:00,GOOG
...,...,...,...,...,...,...,...
1192704,165.010,165.0500,165.000,165.000,213921,2024-04-19 15:56:00-04:00,AAPL
1192705,165.000,165.0500,164.980,165.030,292106,2024-04-19 15:57:00-04:00,AAPL
1192706,165.025,165.0400,164.995,165.010,338342,2024-04-19 15:58:00-04:00,AAPL
1192707,165.005,165.0300,164.890,164.930,1199262,2024-04-19 15:59:00-04:00,AAPL


In [163]:
df= df[_ohlc]
df = df-df[['open']].values
df = df[df['high'] < 5]
df = df[df['close'] > -4]
df = df[df['low'] > -5]
df.head()

,open,high,low,close
475556,0.0,1.2100,-0.02,1.130
475557,0.0,0.1699,-0.19,-0.140
475558,0.0,0.1600,-0.37,0.150
475559,0.0,0.5080,-0.11,0.340
475560,0.0,0.2850,-0.13,-0.112


In [164]:
X = df.values
X.shape

(11704, 4)

In [195]:
# Perform k-means clustering
n_centers = 20
kmeans = KMeans(n_clusters=n_centers, random_state=0).fit(X)
# Print out the cluster centers
centers = kmeans.cluster_centers_
centers

array([[ 0.        ,  0.06294889, -0.36735968, -0.27216206],
       [ 0.        ,  0.52624875, -0.08018565,  0.39316629],
       [ 0.        ,  0.22058782, -0.16493198, -0.03285711],
       [ 0.        ,  0.08754392, -0.94635714, -0.76331164],
       [ 0.        ,  1.2793266 , -0.13485851,  1.06037305],
       [ 0.        ,  3.474     , -0.5986625 ,  3.0233875 ],
       [ 0.        ,  0.54542083, -2.32215   , -1.35794583],
       [ 0.        ,  0.27052676, -0.04495223,  0.19896843],
       [ 0.        ,  0.303605  , -3.104905  , -2.531585  ],
       [ 0.        ,  0.13277867, -1.428888  , -1.20716667],
       [ 0.        ,  0.55024304, -0.27637278, -0.06236013],
       [ 0.        ,  0.85433556, -0.09858533,  0.66409733],
       [ 0.        ,  0.10159024, -0.63569734, -0.48128521],
       [ 0.        ,  0.03538629, -0.05585804, -0.01650547],
       [ 0.        ,  1.49313704, -0.4183    ,  0.11493333],
       [ 0.        ,  2.09089333, -0.26281333,  1.55273   ],
       [ 0.        ,  0.

In [196]:
Xh = kmeans.predict(X)

In [197]:
# Sample OHLC data
df = pd.DataFrame(centers, columns=_ohlc)
# Plot the OHLC chart
# mpf.plot(df, type='candle', style='yahoo', title='OHLC Chart', ylabel='Price')
fig = go.Figure(data=go.Candlestick(x=np.arange(n_centers),
                    open=df['open'],
                    high=df['high'],
                    low=df['low'],
                    close=df['close']))
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()



# OK

In [198]:
Xh

array([ 4,  2, 16, ..., 13, 17, 17], dtype=int32)

In [199]:
X[:,3].mean()

-0.005544338089542167

In [200]:
(X[:,1] > 0.01).mean()

0.8201469583048531

In [208]:
def find_ones_and_previous(numbers):
    result = []
    for i in range(1, len(numbers)):
        if numbers[i] == 6:
            result.append(numbers[i+1])
    return result

def get_ev_pattern(n,numbers):
    result = []
    for i in range(1, len(numbers)):
        if i+1 >= len(numbers):
            continue
        if numbers[i] == n:
            result.append(X[i+1,3])
    return result

def get_1c_pattern(n,numbers):
    result = []
    for i in range(0, len(numbers)-1):
        if numbers[i] == n:
            result.append((X[i+1,1]>0.01))
    return result

def get_1c_scalp(n,numbers):
    result = []
    for i in range(0, len(numbers)-1):
        if numbers[i] == n:
            if X[i+1,1]>0.01:
                result.append(0.02)
            else:
                result.append(X[i+1,3])
    return result

In [210]:
for i in range(n_centers):
    res = np.array(get_ev_pattern(i,Xh))
    print(i,np.mean(res).round(2))

0 -0.01
1 -0.02
2 -0.02
3 0.02
4 0.12
5 -0.37
6 -0.17
7 -0.01
8 -0.51
9 -0.04
10 -0.06
11 0.03
12 -0.03
13 -0.0
14 0.2
15 0.03
16 -0.03
17 -0.0
18 -0.0
19 -0.17


In [211]:
fig = px.histogram(Xh,histnorm='percent',nbins=n_centers).show()

In [222]:
Xl = X[X[:,1]<0.01]
print(Xl.shape)
# Sample OHLC data
df = pd.DataFrame(Xl, columns=_ohlc)
# Plot the OHLC chart
# mpf.plot(df, type='candle', style='yahoo', title='OHLC Chart', ylabel='Price')
fig = go.Figure(data=go.Candlestick(x=np.arange(Xl.shape[0]),
                    open=df['open'],
                    high=df['high'],
                    low=df['low'],
                    close=df['close']))
# fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

(2105, 4)


# Checking patterns for bouncing back, not just immediate next bar being good